In [ ]:
import os
import sys
import numpy as np
import librosa
import pandas as pd
from matplotlib import pyplot as plt
import IPython.display as ipd
from libsoni.util.utils import mix_sonification_and_original

sys.path.append('..')
import libfmp.b
import libfmp.c8

%matplotlib inline

Fs = 22050

In [ ]:
fn_ann = os.path.join('data_csv', '##', 'FMP_B_Sonify_Pitch_Schubert_D911-11_SC06.csv')
fn_wav = os.path.join('data_audio', '##', 'FMP_B_Sonify_Pitch_Schubert_D911-11_SC06.wav')

x, Fs = librosa.load(fn_wav, sr=Fs)
Fs = 22050
x_dur = len(x) / Fs
note_list_df = pd.read_csv(fn_ann, sep=';')
#ipd.display(ipd.HTML(note_list_df[0:10].to_html()))
note_list = [[start, dur, pitch, vel, ins] for i, 
             (start, dur, pitch, vel, ins) in note_list_df.iterrows()]

In [ ]:
# Chromgram from annotations
from libsoni.core.chroma import sonify_chromagram


H = 512 
num_frames = int(len(x) / H)
Fs_frame = Fs / H
C_ann = libfmp.b.list_to_chromagram(note_list, num_frames, Fs_frame)
title = 'Annotation chromagram (Fs_frame = %.3f)' % Fs_frame
libfmp.b.plot_chromagram(C_ann, Fs=Fs_frame, title=title, figsize=(8, 2.5))
plt.show()

In [ ]:
# Sonification
x_chroma_ann = sonify_chromagram(C_ann,H=512,fs=22050, fade_duration=512/22050)
print('Sonification of chroma-based respresentation (mono):')
ipd.display(ipd.Audio(x_chroma_ann, rate=Fs))
print(0.01*Fs)

In [ ]:
ipd.display(ipd.Audio(x_chroma_ann, rate=Fs))
print(0.24*Fs)

In [ ]:
fn_wav = os.path.join('data_audio', '##', '01-AchGottundHerr.wav')

x, Fs = librosa.load(fn_wav, sr=Fs)


import librosa, librosa.display
C = librosa.feature.chroma_stft(y=x, sr=Fs, tuning=0, norm=0, hop_length=512, n_fft=1024)
libfmp.b.plot_chromagram(C, Fs=Fs_frame, title=title, figsize=(8, 2.5))
plt.show()



c_test = sonify_chromagram(C,H=512,fs=22050, fade_duration=0.001)
ipd.display(ipd.Audio(c_test, rate=Fs))
print(len(c_test))
print(C.shape)

In [ ]:
bach_audio, _ = librosa.load(os.path.join('data_audio','demo_f0','01-AchGottundHerr.wav'), sr=Fs)
bach_sonified_w_original = mix_sonification_and_original(sonification=c_test,
                                                         original_audio=bach_audio,
                                                         gain_lin_sonification=1.0,
                                                         gain_lin_original_audio=0.5)
ipd.display(ipd.Audio(bach_sonified_w_original, rate=Fs))


In [ ]:
int((0 - 0.5) * Fs / Fs/H)

In [ ]:
int(0.05*Fs)

In [ ]:
import numpy as np

def linear_smooth_constant_values(arr, fade_values):
    smoothed_arr = np.copy(arr)  # Create a copy of the original array to avoid modifying it in place
    fade_duration = fade_values/Fs
    for i in range(1, len(arr)):
        if arr[i] != arr[i - 1]:
            print('start:',i-int(fade_values/2), 'value:', arr[i-1])
            print('end:',i+int(fade_values/2), 'value:', arr[i])
            
            print('smooted_arr before')
            plt.plot(smoothed_arr[i-int(fade_values/2):i+int(fade_values/2)])
            plt.show()
            
            #smoothed_arr[i-int(fade_values/2):i+int(fade_values/2)] = np.linspace(arr[i-1],arr[i],num=fade_values, dtype=float)
            frequency = 1
            amplitude = (np.abs(arr[i-1]-arr[i])) / 2
            
            
            x = np.linspace(-1* (np.pi / 2), np.pi / 2, fade_values) * -1 * np.sign(arr[i-1]-arr[i])
            
            
            y = amplitude * np.sin(frequency * x) + (arr[i-1] + arr[i]) / 2
            
            
            smoothed_arr[i-int(fade_values/2):i+int(fade_values/2)] =  y
            
            print('smooted_arr after')
            plt.plot(smoothed_arr[i-int(fade_values/2):i+int(fade_values/2)])
            plt.show()
          
            print('linspace')
            plt.plot(np.linspace(arr[i-1],arr[i],num=fade_values, dtype=float))
            plt.show()
        
        
    return smoothed_arr

# Create your original array with constant values for 1000 consecutive indices
original_array = np.array([0.1] * Fs + [1.0] * Fs + [0.3] *Fs)

# Set the number of values to fade between constant values
fade_values = 5000

# Apply linear smoothing
smoothed_array = linear_smooth_constant_values(original_array, fade_values)

# Print the original and smoothed arrays
print("Original Array:")
plt.plot(original_array)
plt.show()
print("\nSmoothed Array:")
plt.plot(smoothed_array)
plt.show()


In [ ]:
shepard = generate_shepard_tone(pitch_class= 0,
                                filter = False,
                                f_center= 440.0,
                                octave_cutoff = 1,
                                gain= 1.0,
                                duration= 3*Fs,
                                fs= 22050,
                                f_tuning= 440,
                                fade_dur= 0) 

ipd.display(ipd.Audio(shepard*original_array, rate=Fs))
ipd.display(ipd.Audio(shepard*smoothed_array, rate=Fs))